In [1]:
import pandas as pd
import numpy as np

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_13276\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Бутстрап

Задача: магазин хочет склонить покупателей к скачиванию мобильного приложения. В качестве эксперимента форму для получения ссылки заменили на кнопку, ведущую в AppStore. На серверах с ID 2 и 3 осталась контрольная версия с формой, на сервере с ID 1 — версия с кнопкой. Проверить, есть ли значимые изменения в средней доле переходов к скачиванию (`VisitPageFlag`)

In [9]:
grocery_data = pd.read_csv("../data/grocery_website.csv")
grocery_data.drop(columns=['LoggedInFlag'])
grocery_data

,RecordID,IP Address,LoggedInFlag,ServerID,VisitPageFlag
0,1,39.13.114.2,1,2,0
1,2,13.3.25.8,1,1,0
2,3,247.8.211.8,1,1,0
3,4,124.8.220.3,0,3,0
4,5,60.10.192.7,0,2,0
...,...,...,...,...,...
184583,184584,114.8.104.1,0,1,0
184584,184585,207.2.110.5,0,2,1
184585,184586,170.13.31.9,0,2,0
184586,184587,195.14.92.3,0,3,0


Некоторые пользователи (идентифицируем их по IP-адресу) заходили на сайт несколько раз, удалите дубликаты пользователей и для каждого заполните `VisitPageFlag=1`, если хотя бы раз для него встречалось значение 1

In [10]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
grocery_data = grocery_data.groupby(["IP Address", "ServerID"])["VisitPageFlag"].max().reset_index()
grocery_data

,IP Address,ServerID,VisitPageFlag
0,0.0.108.2,1,0
1,0.0.109.6,1,0
2,0.0.111.8,3,0
3,0.0.160.9,2,0
4,0.0.163.1,2,0
...,...,...,...
99511,99.9.53.7,2,0
99512,99.9.65.2,2,0
99513,99.9.79.6,2,0
99514,99.9.86.3,1,1


Метрика: отношение количества посещений сайта с `VisitPageFlag=1` к общему количеству посещений.<br>
Для обеих групп сгенерируйте бутстрапированное распределение разностей значения метрик для групп A и B. Не забудьте скорректировать значения и внимательно следите за тем, в какую сторону корректируете.

In [109]:
iterations = 3000
A = grocery_data[grocery_data.ServerID > 1]
B = grocery_data[grocery_data.ServerID == 1]

### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
from tqdm import tqdm

a_entries = len(A)
b_entries = len(B)
boot_data = []
for i in tqdm(range(iterations)):
    a_sample = A.sample(frac = 1, replace = True).VisitPageFlag
    b_sample = B.sample(frac = 1, replace = True).VisitPageFlag
    boot_data.append(b_sample.sum() / b_entries - a_sample.sum() / a_entries)

orig_theta = B.VisitPageFlag.sum() / b_entries - A.VisitPageFlag.sum() / a_entries
boot_theta = np.mean(boot_data)
delta_val = orig_theta - boot_theta
boot_corrected = [i + delta_val for i in boot_data]

100%|██████████| 3000/3000 [04:38<00:00, 10.79it/s]


Найдите границы доверительного интервала и проверьте, значима ли разница значений метрик для A и B (находится ли 0 за пределами интервала)

In [110]:
alpha = 0.05

### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
result_series = pd.Series(boot_corrected)
left_ci = np.quantile(result_series, alpha / 2)
right_ci = np.quantile(result_series, 1 - alpha / 2)
left_ci, right_ci

(0.019320276519543916, 0.027116081964636277)

Вычислите среднее значение скорректированной разности, нам понадобится его запомнить:

In [111]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
result_series.mean()

0.02328062704837657

### Бакетизация

Бутстрапирование на большом объёме данных может происходить очень долго.<br>
* Присвойте каждой записи один из `buckets_count` индексов
* Сгруппируйте в бакеты записи с одинаковыми индексами и группами эксперимента (A/B)
* Для каждого бакета посчитайте сумму `VisitPageFlag` и количество записей

In [42]:
buckets_count = 5000

### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
bucket = np.random.randint(0, buckets_count, size=len(grocery_data))
grocery_data['bucket'] = pd.Series(bucket)
bucketed_A = grocery_data[grocery_data.ServerID > 1]
bucketed_B = grocery_data[grocery_data.ServerID == 1] 

In [99]:
def get_bucket_values(bucketed):
    buckets_sum = pd.Series(bucketed.groupby('bucket').VisitPageFlag.sum()).rename('sum')
    buckets_count = pd.Series(bucketed.groupby('bucket').VisitPageFlag.count()).rename('count')
    buckets = pd.concat([buckets_sum, buckets_count], axis=1)
    return buckets[buckets['count'] > 0]

buckets_A = get_bucket_values(bucketed_A)
buckets_B = get_bucket_values(bucketed_B)

Теперь на полученных выборках повторите процедуру бутстрапа.

In [112]:
iterations = 3000

### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
bucket_boot_data = []
for i in tqdm(range(iterations)):
    a_sample = buckets_A.sample(frac = 1, replace = True)
    b_sample = buckets_B.sample(frac = 1, replace = True)
    bucket_boot_data.append(b_sample['sum'].sum() / b_sample['count'].sum() - a_sample['sum'].sum() / a_sample['count'].sum())

100%|██████████| 3000/3000 [00:30<00:00, 97.92it/s] 


Скорректируйте значения, возьмите среднее и сравните с результатом до бакетизации.<br>

In [118]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
boot_theta = np.mean(bucket_boot_data)
delta_val = orig_theta - boot_theta
bucket_boot_corrected = [i + delta_val for i in bucket_boot_data]
np.mean(bucket_boot_corrected)

0.023280627048376565

Какой можно сделать вывод? Подтверждена или опровергнута $H_0$?

### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ